In [39]:
#import dependencies
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, InputLayer, Dropout, BatchNormalization, Input, Concatenate
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
from keras.regularizers import l2

# Using a two d matrix for indicating home and away team

In [40]:
df = pd.read_csv('nbaPreProcessed.csv')
df

,date,homeTeam,homeTeam_id,homeTeamSubject_id,homeTeam_win%,homeTeam_wins,homeTeam_losses,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,...,awayTeam_Total_Reb,awayTeam_Ast,awayTeam_Stl,awayTeam_Blk,awayTeam_TO,awayTeam_PF,winner,season,isPlayoffGame,winner_binary
0,2024-06-17,Boston Celtics,BOS,2,0.792079,80,21,106,28,39,...,35,18,4,4,13,20,Boston Celtics,2023-2024,True,0
1,2024-06-14,Dallas Mavericks,DAL,7,0.611650,63,40,122,34,27,...,31,18,2,5,13,19,Dallas Mavericks,2023-2024,True,0
2,2024-06-12,Dallas Mavericks,DAL,7,0.607843,62,40,99,31,20,...,36,26,4,6,9,19,Boston Celtics,2023-2024,True,1
3,2024-06-09,Boston Celtics,BOS,2,0.795918,78,20,105,25,29,...,43,21,5,3,15,17,Boston Celtics,2023-2024,True,0
4,2024-06-06,Boston Celtics,BOS,2,0.793814,77,20,107,37,26,...,43,9,8,1,11,16,Boston Celtics,2023-2024,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25726,2004-11-03,Cleveland Cavaliers,CLE,6,0.000000,0,1,104,21,19,...,47,24,9,5,13,28,Indiana Pacers,2004-2005,False,1
25727,2004-11-03,Boston Celtics,BOS,2,0.000000,0,1,95,22,31,...,41,20,9,2,16,26,Philadelphia 76ers,2004-2005,False,1
25728,2004-11-02,Los Angeles Lakers,LAL,14,1.000000,1,0,89,24,26,...,48,17,10,8,16,21,Los Angeles Lakers,2004-2005,False,0
25729,2004-11-02,Detroit Pistons,DET,9,1.000000,1,0,87,19,18,...,36,8,4,5,16,24,Detroit Pistons,2004-2005,False,0


In [41]:
#populate dfIDtoStat - should be double the size of df
homedf = df[['date', 'homeTeamSubject_id', 'homeTeam_win%', 'homeTeam_wins', 'homeTeam_losses', 'homeTeam_points_total', 'homeTeam_points_q1', 'homeTeam_points_q2', 'homeTeam_points_q3', 'homeTeam_points_q4', 'homeTeam_points_1OT', 'homeTeam_points_2OT', 'homeTeam_points_3OT', 'homeTeam_points_4OT', 'homeTeam_FG', 'homeTeam_FG_made', 'homeTeam_3P', 'homeTeam_3P_made', 'homeTeam_O_Reb', 'homeTeam_D_Reb', 'homeTeam_Total_Reb', 'homeTeam_Ast', 'homeTeam_Stl', 'homeTeam_Blk', 'homeTeam_TO', 'homeTeam_PF']]
homedf.columns = ['date', 'team_id', 'team_win%', 'team_wins', 'team_losses', 'team_points_total', 'team_points_q1', 'team_points_q2', 'team_points_q3', 'team_points_q4', 'team_points_1OT', 'team_points_2OT', 'team_points_3OT', 'team_points_4OT', 'team_FG', 'team_FG_made', 'team_3P', 'team_3P_made', 'team_O_Reb', 'team_D_Reb', 'team_Total_Reb', 'team_Ast', 'team_Stl', 'team_Blk', 'team_TO', 'team_PF']
awaydf = df[['date', 'awayTeamSubject_id', 'awayTeam_win%', 'awayTeam_wins', 'awayTeam_losses', 'awayTeam_points_total', 'awayTeam_points_q1', 'awayTeam_points_q2', 'awayTeam_points_q3', 'awayTeam_points_q4', 'awayTeam_points_1OT', 'awayTeam_points_2OT', 'awayTeam_points_3OT', 'awayTeam_points_4OT', 'awayTeam_FG', 'awayTeam_FG_made', 'awayTeam_3P', 'awayTeam_3P_made', 'awayTeam_O_Reb', 'awayTeam_D_Reb', 'awayTeam_Total_Reb', 'awayTeam_Ast', 'awayTeam_Stl', 'awayTeam_Blk', 'awayTeam_TO', 'awayTeam_PF']]
awaydf.columns = ['date', 'team_id', 'team_win%', 'team_wins', 'team_losses', 'team_points_total', 'team_points_q1', 'team_points_q2', 'team_points_q3', 'team_points_q4', 'team_points_1OT', 'team_points_2OT', 'team_points_3OT', 'team_points_4OT', 'team_FG', 'team_FG_made', 'team_3P', 'team_3P_made', 'team_O_Reb', 'team_D_Reb', 'team_Total_Reb', 'team_Ast', 'team_Stl', 'team_Blk', 'team_TO', 'team_PF']

dfIDtoStat = pd.concat([homedf, awaydf], ignore_index=True)
dfIDtoStat

,date,team_id,team_win%,team_wins,team_losses,team_points_total,team_points_q1,team_points_q2,team_points_q3,team_points_q4,...,team_3P,team_3P_made,team_O_Reb,team_D_Reb,team_Total_Reb,team_Ast,team_Stl,team_Blk,team_TO,team_PF
0,2024-06-17,2,0.792079,80,21,106,28,39,19,20,...,0.333333,13,15,36,51,25,9,2,7,15
1,2024-06-14,7,0.611650,63,40,122,34,27,31,30,...,0.405405,15,13,39,52,21,7,2,8,17
2,2024-06-12,7,0.607843,62,40,99,31,20,19,29,...,0.360000,9,7,36,43,15,5,1,8,17
3,2024-06-09,2,0.795918,78,20,105,25,29,29,22,...,0.256410,10,10,31,41,29,10,5,10,15
4,2024-06-06,2,0.793814,77,20,107,37,26,23,21,...,0.380952,16,10,37,47,23,6,9,12,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51457,2004-11-03,12,1.000000,1,0,109,26,20,17,22,...,0.235294,4,11,36,47,24,9,5,13,28
51458,2004-11-03,23,1.000000,1,0,98,27,24,20,27,...,0.375000,9,14,27,41,20,9,2,16,26
51459,2004-11-02,8,0.000000,0,1,78,14,17,26,21,...,0.250000,3,16,32,48,17,10,8,16,21
51460,2004-11-02,11,0.000000,0,1,79,18,20,21,20,...,0.461538,6,9,27,36,8,4,5,16,24


In [30]:
one_hot = pd.get_dummies(dfIDtoStat['team_id'], prefix='team',  dtype=int)
dfIDtoStat = pd.concat([dfIDtoStat, one_hot], axis=1)

dfIDtoStat

,date,team_id,team_win%,team_wins,team_losses,team_points_total,team_points_q1,team_points_q2,team_points_q3,team_points_q4,...,team_21,team_22,team_23,team_24,team_25,team_26,team_27,team_28,team_29,team_30
0,2024-06-17,2,0.792079,80,21,106,28,39,19,20,...,0,0,0,0,0,0,0,0,0,0
1,2024-06-14,7,0.611650,63,40,122,34,27,31,30,...,0,0,0,0,0,0,0,0,0,0
2,2024-06-12,7,0.607843,62,40,99,31,20,19,29,...,0,0,0,0,0,0,0,0,0,0
3,2024-06-09,2,0.795918,78,20,105,25,29,29,22,...,0,0,0,0,0,0,0,0,0,0
4,2024-06-06,2,0.793814,77,20,107,37,26,23,21,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51457,2004-11-03,12,1.000000,1,0,109,26,20,17,22,...,0,0,0,0,0,0,0,0,0,0
51458,2004-11-03,23,1.000000,1,0,98,27,24,20,27,...,0,0,1,0,0,0,0,0,0,0
51459,2004-11-02,8,0.000000,0,1,78,14,17,26,21,...,0,0,0,0,0,0,0,0,0,0
51460,2004-11-02,11,0.000000,0,1,79,18,20,21,20,...,0,0,0,0,0,0,0,0,0,0


In [31]:
windowConditions = ['team_points_total', 'team_points_q1', 'team_points_q2', 'team_points_q3', 'team_points_q4', 'team_points_1OT', 'team_points_2OT', 'team_points_3OT', 'team_points_4OT', 'team_FG', 'team_FG_made', 'team_3P', 'team_3P_made', 'team_O_Reb', 'team_D_Reb', 'team_Total_Reb', 'team_Ast', 'team_Stl', 'team_Blk', 'team_TO', 'team_PF']
print(windowConditions)


['team_points_total', 'team_points_q1', 'team_points_q2', 'team_points_q3', 'team_points_q4', 'team_points_1OT', 'team_points_2OT', 'team_points_3OT', 'team_points_4OT', 'team_FG', 'team_FG_made', 'team_3P', 'team_3P_made', 'team_O_Reb', 'team_D_Reb', 'team_Total_Reb', 'team_Ast', 'team_Stl', 'team_Blk', 'team_TO', 'team_PF']


In [32]:
#generate the X Y matrices
def df_to_X_y(df, dfWindow, window_size):
    X = []
    y = []
    #each team must have played window_size games before data can be extracted
    for index, row in df.iterrows():
        homeTeam_id = row['homeTeamSubject_id']
        awayTeam_id = row['awayTeamSubject_id']
        date = row['date']
        #limit dfWindow to include rows that occured before date of row we are on
        dfWindow_before_date = dfWindow[dfWindow['date']< date]
        homeTeam_occurrences = dfWindow_before_date[dfWindow_before_date['team_id'] == homeTeam_id].shape[0]
        awayTeam_occurrences = dfWindow_before_date[dfWindow_before_date['team_id'] == awayTeam_id].shape[0]
        if homeTeam_occurrences > window_size and awayTeam_occurrences > window_size:
            homeTeam_window = (dfWindow_before_date[dfWindow_before_date['team_id'] == homeTeam_id].sort_values(by='date', ascending=False).head(window_size).drop(columns=['date', 'team_id']))
            awayTeam_window = (dfWindow_before_date[dfWindow_before_date['team_id'] == awayTeam_id].sort_values(by='date', ascending=False).head(window_size).drop(columns=['date', 'team_id']))
            #condtional for average or top entry - home Team
            true_cols = []
            false_cols = []
            for val in windowConditions:
                true_cols.append(val)
            false_cols = [col for col in homeTeam_window.columns if col not in true_cols]
            homeTeam_window = np.concatenate([homeTeam_window[false_cols].iloc[0].to_numpy(), homeTeam_window[true_cols].mean(axis=0).to_numpy()])
            #conditional for average or top entry - away Team
            awayTeam_window = np.concatenate([awayTeam_window[false_cols].iloc[0].to_numpy(), awayTeam_window[true_cols].mean(axis=0).to_numpy()])
            combined_window = np.vstack((homeTeam_window, awayTeam_window))
            X.append(combined_window)
            y.append(row['winner_binary'])
    
    return np.array(X), np.array(y)

In [33]:
#X1.Shape is num data, window size, variables
X, y = df_to_X_y(df, dfIDtoStat, 20)
X.shape, y.shape

((25406, 2, 54), (25406,))

In [34]:
#80-10-10 split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((20324, 2, 54), (20324,), (2541, 2, 54), (2541,), (2541, 2, 54), (2541,))

In [42]:
input_layer = Input(shape=(2, 54))
home_team_input = input_layer[:, 0, :]
away_team_input = input_layer[:, 1, :]

home_dense = Dense(64, activation='relu')(home_team_input)
away_dense = Dense(64, activation='relu')(away_team_input)

combined = Concatenate()([home_dense, away_dense])

x = Dense(32, activation='relu')(combined)
x = Dense(16, activation='relu')(x)
x = Dense(8, activation='relu')(x)
x = Dense(4, activation='relu')(x)

output_layer = Dense(1, activation='sigmoid')(x)

model = Model(inputs=input_layer, outputs=output_layer)

In [43]:
#early stopping if model does not improve performance for 50 epochs
earlystopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [44]:
#simple model fit
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, callbacks=[earlystopping])

Epoch 1/100
636/636 [==============================] - 2s 2ms/step - loss: 0.7403 - accuracy: 0.6093 - val_loss: 0.6565 - val_accuracy: 0.6482
Epoch 2/100
636/636 [==============================] - 1s 1ms/step - loss: 0.6625 - accuracy: 0.6190 - val_loss: 0.6496 - val_accuracy: 0.6395
Epoch 3/100
636/636 [==============================] - 1s 1ms/step - loss: 0.6450 - accuracy: 0.6381 - val_loss: 0.7190 - val_accuracy: 0.4557
Epoch 4/100
636/636 [==============================] - 1s 1ms/step - loss: 0.6401 - accuracy: 0.6415 - val_loss: 0.6319 - val_accuracy: 0.6545
Epoch 5/100
636/636 [==============================] - 1s 1ms/step - loss: 0.6339 - accuracy: 0.6434 - val_loss: 0.6297 - val_accuracy: 0.6576
Epoch 6/100
636/636 [==============================] - 1s 1ms/step - loss: 0.6295 - accuracy: 0.6501 - val_loss: 0.6233 - val_accuracy: 0.6556
Epoch 7/100
636/636 [==============================] - 1s 1ms/step - loss: 0.6303 - accuracy: 0.6469 - val_loss: 0.6167 - val_accuracy: 0.6643

In [45]:
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)
print(accuracy_score(y_test, y_pred_binary))

80/80 [==============================] - 0s 684us/step
0.6654860291223927
